In [36]:
import torch
from numpy import asarray,stack,concatenate,convolve,ones,arange,stack,log,histogram,maximum
from glob import glob
from XRDXRFutils import DataXRF, SyntheticDataXRF

from matplotlib.pyplot import plot,semilogx,semilogy,subplots,legend,fill_between,xlim,ylim,imshow,subplots, suptitle
import re

In [37]:
def read_images(path):
    checkpoints = sorted(glob(path + '/*.pth'))

    images = []
    for checkpoint in checkpoints:
        images += [torch.load(checkpoint,map_location=torch.device('cpu'))['image']]

    return images

In [ ]:
images = read_images('../run/CNN2_6_ch_run_0/')
data_elgreco = DataXRF().load_h5('F:/andrea/AIdataXRF/base/ElGrecoAnalysis_downsample66_weighted_x1024_with_labels.h5')
data_giulia = DataXRF().load_h5('/home/shared/AIdataXRF/downsample6_giulia_weighted_1024_with_labels.h5')
data_henry = DataXRF().load_h5('/home/shared/AIdataXRF/downsample9_henryVII_x1024_with_labels.h5')

In [38]:
images = read_images('../run/synth_scales_0,1-1,1_CNN2L1Loss11_ch_run_0/')

In [39]:
selected_labels = ['Pb-L','Au-L','Ca-K','Co-K','Cu-K','Fe-K','Mn-K','Hg-L', 'Ti-K','Zn-K']
data_elgreco = DataXRF().load_h5('F:/andrea/AIdataXRF/base/ElGrecoAnalysis_downsample66_weighted_x1024_with_labels.h5')
data_elgreco.select_labels(selected_labels)
data_henry = DataXRF().load_h5('F:/andrea/AIdataXRF/base/3317HVIIAnalysisBis_downsample99_weighted_x1024_with_labels.h5')
data_henry.select_labels(selected_labels)
data_henry8 = DataXRF().load_h5('F:/andrea/AIdataXRF/base/3314MargaretAnalysis_downsample1111_weighted_x1024_with_labels.h5')
data_henry8.select_labels(selected_labels)

Loading: F:/andrea/AIdataXRF/base/ElGrecoAnalysis_downsample66_weighted_x1024_with_labels.h5
Adding empty labels: ['Au-L']
Loading: F:/andrea/AIdataXRF/base/3317HVIIAnalysisBis_downsample99_weighted_x1024_with_labels.h5
Adding empty labels: ['Co-K']
Loading: F:/andrea/AIdataXRF/base/3314MargaretAnalysis_downsample1111_weighted_x1024_with_labels.h5
Adding empty labels: ['Ti-K']


In [16]:
giulia, elgreco, henry = images[-1]
giulia.shape,elgreco.shape,henry.shape

((68, 121, 8), (82, 71, 8), (90, 65, 8))

In [40]:
elgreco, henry, henry8, _ = images[-1]
henry.shape,elgreco.shape,henry8.shape

((82, 71, 10), (68, 121, 10), (90, 65, 10))

In [310]:
%matplotlib qt
fig,ax = subplots(3,10,figsize=(19,8))
for i in range(5):

    ax[0,2*i].imshow(margaret[:,:,i],vmin=0,vmax = margaret[:,:,i].max())
    ax[1,2*i].imshow(elgreco[:,:,i],vmin=0,vmax = elgreco[:,:,i].max())
    ax[2,2*i].imshow(henry[:,:,i],vmin=0,vmax = henry[:,:,i].max())

    ax[1,2*i+1].imshow(data_elgreco.labels[:,:,i])
    ax[0,2*i+1].imshow(data_margaret.labels[:,:,i])
    ax[2,2*i+1].imshow(data_henry.labels[:,:,i])

In [41]:
%matplotlib qt
cmap = 'pink'
fig,ax = subplots(3,2*len(selected_labels),figsize=(19,8))
for i in range(len(selected_labels)):
    henry8_max = max(henry8[:,:,i].max(), data_henry8.labels[:,:,i].max())
    elgreco_max = max(elgreco[:,:,i].max(), data_elgreco.labels[:,:,i].max())
    henry_max = max(henry[:,:,i].max(), data_henry.labels[:,:,i].max())

    ax[0,2*i].imshow(henry8[:,:,i],vmin=0,vmax = henry8_max, cmap=cmap)
    ax[0,2*i].set_title(selected_labels[i]+': CNN')
    ax[1,2*i].imshow(elgreco[:,:,i],vmin=0,vmax = elgreco_max, cmap=cmap)
    ax[1,2*i].set_title(selected_labels[i]+': CNN')
    ax[2,2*i].imshow(henry[:,:,i],vmin=0,vmax = henry_max, cmap=cmap)
    ax[2,2*i].set_title(selected_labels[i]+': CNN')

    ax[0,2*i+1].imshow(data_henry8.labels[:,:,i],vmin=0,vmax = henry8_max, cmap=cmap)
    ax[0,2*i+1].set_title('GT')
    ax[1,2*i+1].imshow(data_elgreco.labels[:,:,i],vmin=0,vmax = elgreco_max, cmap=cmap)
    ax[1,2*i+1].set_title('GT')
    ax[2,2*i+1].imshow(data_henry.labels[:,:,i],vmin=0,vmax = henry_max, cmap=cmap)
    ax[2,2*i+1].set_title('GT')



In [50]:
for element, label in enumerate(selected_labels):
    fig, ax = subplots(1,2)
    ax[0].set_title('model '+selected_labels[element])
    ax[1].set_title('MA-XRF '+selected_labels[element])
    ax[0].imshow(henry[:,:,element],vmin=0,vmax = max(henry[:,:,element].max(), data_henry.labels[:,:,element].max()), cmap=cmap)
    ax[1].imshow(data_henry.labels[:,:,element], vmax = max(henry[:,:,element].max(), data_henry.labels[:,:,element].max()),cmap=cmap)

In [51]:
%matplotlib qt
elgreco_labels = data_elgreco.labels.reshape(-1,len(selected_labels))
henry8_labels = data_henry8.labels.reshape(-1,len(selected_labels))
henry_labels = data_henry.labels.reshape(-1,len(selected_labels))

elgreco_predictions = elgreco.reshape(-1,len(selected_labels))
henry8_predictions = henry8.reshape(-1,len(selected_labels))
henry_predictions = henry.reshape(-1,len(selected_labels))

fig,ax = subplots(len(selected_labels),3,figsize=(12,12))
suptitle('HISTOGRAMS')
ax[0,0].set_title('henry8')
ax[0,1].set_title('elgreco')
ax[0,2].set_title('henry')

for i in range(len(selected_labels)):
    m = maximum(henry8_labels.max(axis=0),henry8_predictions.max(axis=0))
    
    ry,rx = histogram(henry8_labels[:,i],bins=int(m[i]+1),range=(0,m[i]+1))
    iy,ix = histogram(henry8_predictions[:,i],bins=int(m[i]+1),range=(0,m[i]+1))

    ax[i,0].plot(rx[:-1],ry,label=selected_labels[i]+' - labels')
    ax[i,0].plot(ix[:-1],iy,label=selected_labels[i]+' - model', alpha=0.8)
    ax[i,0].legend()
    
    m = maximum(elgreco_labels.max(axis=0),elgreco_predictions.max(axis=0))
                
    ry,rx = histogram((elgreco_labels[:,i]),bins=int(m[i]+1),range=(0,m[i]+1))
    iy,ix = histogram((elgreco_predictions[:,i]),bins=int(m[i]+1),range=(0,m[i]+1))

    ax[i,1].plot(rx[:-1],ry,label=selected_labels[i]+' - labels')
    ax[i,1].plot(ix[:-1],iy,label=selected_labels[i]+' - model', alpha=0.8)
    ax[i,1].legend()
    
    m = maximum(henry_labels.max(axis=0),henry_predictions.max(axis=0))
    
    ry,rx = histogram(henry_labels[:,i],bins=int(m[i]+1),range=(0,m[i]+1))
    iy,ix = histogram(henry_predictions[:,i],bins=int(m[i]+1),range=(0,m[i]+1))

    ax[i,2].plot(rx[:-1],ry,label=selected_labels[i]+' - labels')
    ax[i,2].plot(ix[:-1],iy,label=selected_labels[i]+' - model', alpha=0.8)
    ax[i,2].legend()

In [ ]:
%matplotlib inline
elgreco_labels = data_elgreco.labels.reshape(-1,5)
giulia_labels = data_giulia.labels.reshape(-1,5)
henry_labels = data_henry.labels.reshape(-1,5)

img_elgreco_labels = elgreco.reshape(-1,5)
img_giulia_labels = giulia.reshape(-1,5)
img_henry_labels = henry.reshape(-1,5)

fig,ax = subplots(5,3,figsize=(12,12))

for i in range(5):
    m = maximum(giulia_labels.max(axis=0),img_giulia_labels.max(axis=0))
    
    ry,rx = histogram(giulia_labels[:,i],bins=100,range=(0,m[i]))
    iy,ix = histogram(img_giulia_labels[:,i],bins=100,range=(0,m[i]))

    ax[i,0].plot(rx[:-1],ry)
    ax[i,0].plot(ix[:-1],iy)
    
    m = maximum(elgreco_labels.max(axis=0),img_elgreco_labels.max(axis=0))
                
    ry,rx = histogram((elgreco_labels[:,i]),bins=100,range=(0,m[i]))
    iy,ix = histogram((img_elgreco_labels[:,i]),bins=100,range=(0,m[i]))

    ax[i,1].plot(rx[:-1],ry)
    ax[i,1].plot(ix[:-1],iy)
    
    m = maximum(henry_labels.max(axis=0),img_henry_labels.max(axis=0))
    
    ry,rx = histogram(henry_labels[:,i],bins=100,range=(0,m[i]))
    iy,ix = histogram(img_henry_labels[:,i],bins=100,range=(0,m[i]))

    ax[i,2].plot(rx[:-1],ry)
    ax[i,2].plot(ix[:-1],iy)

In [13]:
dss = []
data_elgreco = DataXRF().load_h5('F:/andrea/AIdataXRF/base/2156HVIIAnalysisBisShiftCorrected_$_weighted_x1024_with_labels.h5')
dss.append(data_elgreco.select_labels(selected_labels))

data_elgreco = DataXRF().load_h5('F:/andrea/AIdataXRF/base/3314MargaretAnalysis_weighted_x1024_with_labels.h5')
dss.append(data_elgreco.select_labels(selected_labels))

data_elgreco = DataXRF().load_h5('F:/andrea/AIdataXRF/base/3317HVIIAnalysisBis_weighted_x1024_with_labels.h5')
dss.append(data_elgreco.select_labels(selected_labels))


Loading: F:/andrea/AIdataXRF/base/2156HVIIAnalysisBisShiftCorrected_$_weighted_x1024_with_labels.h5
Adding empty labels: ['Co-K', 'Ti-K']
Loading: F:/andrea/AIdataXRF/base/3314MargaretAnalysis_weighted_x1024_with_labels.h5
Adding empty labels: ['Ti-K']
Loading: F:/andrea/AIdataXRF/base/3317HVIIAnalysisBis_weighted_x1024_with_labels.h5
Adding empty labels: ['Co-K']


In [487]:
%matplotlib qt
fig, ax = subplots(len(selected_labels),1)
for i in range(len(selected_labels)):
    for idxds,ds in enumerate(dss):
        m = ds[:,:,i].max()
            
        ry,rx = histogram(ds.reshape((-1,len(selected_labels)))[:,i],bins=3000,range=(0,3000))

        ax[i].plot(rx[:-1],ry,label='ds '+str(idxds))
        ax[i].legend()